In [1]:
from base_classification import *

In [2]:
from __future__ import print_function
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import datasets
from sklearn import svm
# for sampling
import random # use random.choice? and random.sample for interactions

from sklearn.metrics import make_scorer
from sklearn.datasets import make_classification
from sklearn.metrics import *


In [3]:
X, y = make_classification(2000)

In [4]:
# start...

In [5]:
bmars = BMARS(X)

In [6]:
alpha, model1 = mh_iter(X, y, 2, bmars)

action: birth
output: {'basis': [1, 3], 'knot': array([ 0.59436498,  0.87546784]), 'sign': [1, -1]}
acceptance_proba: 1.0
	bayes_factor: 1.01083320057
	prior_ratio: 7.00333539552e-09
	proposal_ratio: 428367317.938


In [7]:
alpha, model1 = mh_iter(X, y, 2, bmars, debug=False)

In [8]:
model1

{'X': array([[-0.58016578,  0.92775034,  1.31247434,  1.07849959,  0.63140012,
          0.08414036, -1.83135211, -0.57024664, -1.56810233,  0.21853767,
         -0.1415859 ,  1.33983784, -0.88527229, -0.057768  ,  0.57796127,
          0.7276254 , -1.11872024, -0.85011918, -1.47461721,  0.74588238]]),
 'basis': [[2, 5]],
 'interaction': 2,
 'params': [{'knots': array([-0.4737089 , -1.14185145]), 'signs': [1, -1]}]}